In [1]:
from duckduckgo_search import DDGS

In [ ]:
# import os
# from typing import Any, Text, Dict, List
# from duckduckgo_search import DDGS  # pip install duckduckgo-search
# from openai import OpenAI  # pip install openai

# from rasa_sdk import Action, Tracker
# from rasa_sdk.executor import CollectingDispatcher
# from rasa_sdk.events import SlotSet


class ActionGiveCareerAdvice(Action):
    def name(self) -> Text:
        return "action_give_career_advice"

    def run(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict[Text, Any]]:

        # Get slots
        major = tracker.get_slot("current_major") or "your major"
        interest = tracker.get_slot("career_interest") or "careers"
        year = tracker.get_slot("year_of_study") or "your year"
        gpa = tracker.get_slot("gpa") or "strong performance"  # Optional
        has_internship = tracker.get_slot("has_internship") or False

        # Build search query (contextual + timely)
        query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"
        
        try:
            # Search DuckDuckGo (top 3 results)
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=3))
            
            # Extract snippets & sources
            snippets = [r["body"] for r in results]
            sources = [r["href"] for r in results]
            sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
            
            # LLM prompt for friendly summary
            client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            prompt = f"""
            You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
            Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
            Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
            
            Snippets:
            {chr(10).join(snippets)}
            """
            
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # Cheap & fast
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=300
            )
            advice_text = response.choices[0].message.content.strip()
            
        except Exception as e:
            # Fallback if search/LLM fails
            advice_text = f"""
            Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
            - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
            - **Build skills** with free Coursera courses in Python or data viz.
            - **Internships**: Check Handshake for summer spots; start applying now.
            - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
            Remember, every expert started as a student like you! 💪
            """
            sources_list = "Fallback: General career resources (e.g., your school's site)."
        
        # Utter advice
        dispatcher.utter_message(
            response="utter_advice_core",
            advice_text=advice_text,
            current_major=major,
            career_interest=interest,
            year_of_study=year,
        )
        dispatcher.utter_message(response="utter_encourage_human")
        dispatcher.utter_message(
            response="utter_cite_sources",
            sources_list=sources_list
        )
        
        return []

In [2]:
# Get slots
major = "your major"
interest = "careers"
year = "your year"
gpa = "strong performance"  # Optional
has_internship = False

# Build search query (contextual + timely)
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

try:
    # Search DuckDuckGo (top 3 results)
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    
    # Extract snippets & sources
    snippets = [r["body"] for r in results]
    sources = [r["href"] for r in results]
    sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
    
    # # LLM prompt for friendly summary
    # client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # prompt = f"""
    # You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
    # Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
    # Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
    
    # Snippets:
    # {chr(10).join(snippets)}
    # """
    
    # response = client.chat.completions.create(
    #     model="gpt-4o-mini",  # Cheap & fast
    #     messages=[{"role": "user", "content": prompt}],
    #     temperature=0.3,
    #     max_tokens=300
    # )
    # advice_text = response.choices[0].message.content.strip()
    
except Exception as e:
    # Fallback if search/LLM fails
    advice_text = f"""
    Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
    - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
    - **Build skills** with free Coursera courses in Python or data viz.
    - **Internships**: Check Handshake for summer spots; start applying now.
    - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
    Remember, every expert started as a student like you! 💪
    """
    sources_list = "Fallback: General career resources (e.g., your school's site)."

# # Utter advice
# dispatcher.utter_message(
#     response="utter_advice_core",
#     advice_text=advice_text,
#     current_major=major,
#     career_interest=interest,
#     year_of_study=year,
# )
# dispatcher.utter_message(response="utter_encourage_human")
# dispatcher.utter_message(
#     response="utter_cite_sources",
#     sources_list=sources_list
# )

/tmp/ipykernel_21474/1277786559.py:13: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [7]:
from ddgs import DDGS

# Perform a text search
results = DDGS().text("python programming", max_results=5)


snippets = [r["body"] for r in results]
sources = [r["href"] for r in results]
sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])

print(snippets)
print(sources_list)

['Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Beginning with Python 3.5, capabilities and keywords for typing were added to language, allowing optional static typing. Currently only versions in the 3.x series are supported. Python has gained widespread use in the machine learning community. It is widely taught as an introductory programming language. Since 2003, Python has consistently ranked in the top ten of the most popular programming languages in the TIOBE Programming 

In [9]:
from ddgs import DDGS

# Example variables
interest = "data science"
major = "computer science"
year = "junior"
gpa = "3.8"
has_internship = True

# Build query
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

# Perform search
results = DDGS().text(query, max_results=3)
for result in results:
    print(result["body"], result["href"])   

Start your journey to a rewarding data career with our expert-led training program! Your journey to a successful data career starts here—enroll in our expert-led training! https://duckduckgo.com/y.js?ad_domain=newtotraining.com&ad_provider=bingv7aa&ad_type=txad&click_metadata=V81_D_7A20GJPOqMJthVDKg5ptudQ_aU5EqI7cc663ltfErwod1DT3YunE-SqXPu7IfsDrHEOVSmc2uYb6WBlvXh0ak7vyDDwomrTsbug8uwTBeYqxmbI7EC32eE_gKBE80JVh524R_SWguA6TjRWg.y9sFYxZSQZ6O8VCavzohIQ&rut=164355feaab35c265428f6fc6fff318a4b70e7ef01938a2ca659f053103a9ff4&u3=https://www.bing.com/aclick?ld=e8oqqLRnYsKDgGYV2CdVX7HzVUCUzzv2nFf1_VTVWPxG1xT9ZVwi9Blziio8cp6MkTonwEY_Nqp16PWfGe8wcvKhjnJPZMj0m4I-O9XlhVHHr7z4tV5fNuH_XSHqTNJQ5Qck1XJGSetRr7U7wXNLTMbPtj_JjFQ2rLvDbPuxX0sGY1hTZbtV5096HtSpyr8KMDgTS9wsV-NMD1CzLV3baa9m8O_Mo&u=aHR0cHMlM2ElMmYlMmZhcHBseS5uZXd0b3RyYWluaW5nLmNvbSUyZmRhdGEtYW5hbHlzdC1wcm9ncmFtbWUtYmluZy1hcHBsaWNhdGlvbiUyZiUzZm1zY2xraWQlM2Q0OWI3MDcyNzEwNzYxODNlZTMxMDZlMmVjNmRiZDkyZg&rlid=49b707271076183ee3106e2ec6dbd92f&vqd=4-8206874

In [ ]:
from duckduckgo_ai_chat import DuckDuckGoAIChat

chat = DuckDuckGoAIChat()
response = chat.ask("What are the latest trends in AI?")
print(response)

/workspaces/ai-assistant/.venv/lib/python3.12/site-packages/pygments/token.py:24: ResourceWarning: unclosed <ssl.SSLSocket fd=86, family=2, type=1, proto=6, laddr=('10.0.1.155', 35256), raddr=('40.114.177.156', 443)>
  def __init__(self, *args):


ImportError: cannot import name 'DuckDuckGoAIChat' from 'duckduckgo_ai_chat' (/workspaces/ai-assistant/.venv/lib/python3.12/site-packages/duckduckgo_ai_chat/__init__.py)